In [ ]:
import pytraj as pt
import numpy as np

In [ ]:
'''load the pdb file you will want to calculate for'''
pdb = pt.load('./13mer.pdb')

In [ ]:
'''load standard tC0 used to mimic the DA pairs'''
tc=pt.load('./tC0.pdb')

In [ ]:
'''load the standard dC in case the residue at the postion for DA is not a dC'''
dc=pt.load('./dC_bdna.pdb')

In [ ]:
'''fist superpostion of the standard dC to the residue and then tC0 to the dC'''
pt.rmsd(dc,mask="@P,OP*,C1',C2',C3',C4',C5',O3',O5'",ref=pdb,ref_mask=":4@P,OP*,C1',C2',C3',C4',C5',O3',O5'",update_coordinate=True)
pt.rmsd(tc,mask='@N1,C2,N3,C4,C5,C6',ref=dc,ref_mask='@N1,C2,N3,C4,C5,C6',update_coordinate=True)

array([34.47910629, 41.34136377, 44.77953178])

In [ ]:
'''calculate the center of geometry for the middle ring of tC0'''
dcog = pt.center_of_geometry(tc,mask='@C4,C5,N7,C8,C13,O14')
dcog[0]

array([31.33391073, 39.48269154, 45.29973793])

In [ ]:
'''save the coords to output array'''
output = np.vstack((dcog,tc['@C9'][0][0]))
output

array([[31.33391073, 39.48269154, 45.29973793],
       [28.63156217, 38.98746655, 45.64356278]])

In [ ]:
'''superposition of tC0 mimic to the dC residue position'''
rmsd = pt.rmsd(tc,mask='@N1,C2,N3,C4,C5,C6',ref=pdb,ref_mask=':18@N1,C2,N3,C4,C5,C6',update_coordinate=True)
tc[0][0]

array([36.06682217, 38.16370878, 21.63025543])

In [ ]:
'''calculate the center of geometry for the middle ring of the tC0 mimic'''
pcog = pt.center_of_geometry(tc,mask='@C4,C5,N7,C8,C13,O14')
pcog[0]

array([34.94789744, 34.65453066, 21.85654829])

In [ ]:
'''save output array'''
output = np.vstack((output,pcog[0],tc['@C12'][0][0],dcog[0],pcog[0]))

In [ ]:
output

array([[31.33391073, 39.48269154, 45.29973793],
       [28.63156217, 38.98746655, 45.64356278],
       [34.94789744, 34.65453066, 21.85654829],
       [32.61704246, 33.17396727, 21.61940873],
       [31.33391073, 39.48269154, 45.29973793],
       [34.94789744, 34.65453066, 21.85654829]])

In [ ]:
'''write the output coord to local drive for record keeping'''
np.savetxt("fret_13mer_coord.txt",output,delimiter="\t")

In [ ]:
'''
Calculation for the E value and tDA fluorescence lifetime value.
tD should be set at the value of fluorescence lifetime in abscense of acceptor
'''
tD = 4.5
p1 = output[1,:]-output[0,:]
p2 = output[3,:]-output[2,:]
p3 = output[5,:]-output[4,:]
e1 = np.dot(p1,p2)/(np.linalg.norm(p1)*np.linalg.norm(p2))
e2 = np.dot(p1,p3)/(np.linalg.norm(p1)*np.linalg.norm(p3))
e3 = np.dot(p3,p2)/(np.linalg.norm(p3)*np.linalg.norm(p2))
k = e1 - 3*e2*e3
r0 = 0.211*np.power((k**2*0.23*1.3*10**14/1.4**4),(1/6))
r = np.linalg.norm(pcog-dcog)
e_val = r0**6/(r0**6+r**6)
t_val =(1-e_val)*tD
print('e = {} tDA = {}'.format(e_val,t_val))


e = 0.754919876901423 tDA = 1.1028605539435963
